In [2]:
import sys
from pathlib import Path
from mlx_lm import generate, utils

def _find_project_root(start: Path = Path.cwd()):
	for p in [start] + list(start.parents):
		if (p / "pyproject.toml").exists():
			return p
	return start

project_root = _find_project_root()
sys.path.insert(0, str(project_root))

from load_data import LoadData
from lora.core import LORA, FUSE

In [3]:
# Configuration

root_folder = "../.cache/model_1"

data_folder = f"./{root_folder}/data"
dataset_name = "b-mc2/sql-create-context"
n = 150
test_split_ratio = 0.2
valid_split_ratio = 0.2

model_path = "mistralai/Mistral-7B-Instruct-v0.2"
adapter_file = f"./{root_folder}/adapters.npz"
save_model_path = f"./{root_folder}/model"

In [4]:
# Create Train, Test and Validation Data from the Dataset

system_message = """You are an text to SQL query translator. Users will ask you questions in English and you will generate a SQL query based on the provided SCHEMA.
SCHEMA:
{schema}"""

def create_conversation(input: dict) -> dict:
    if input['question'] is None or input["answer"] is None or input["context"] is None:
        pass
    return {
        "messages": [
            {"role": "system", "content": system_message.format(schema=input["context"])},
            {"role": "user", "content": input["question"]},
            {"role": "assistant", "content": input["answer"]}
        ]
    }

data_loader = LoadData(folder=data_folder, dataset_name=dataset_name)
data_loader.save(function=create_conversation, n=n, test_split_ratio=test_split_ratio, valid_split_ratio=valid_split_ratio, write_files=True)

Map:   0%|          | 0/150 [00:00<?, ? examples/s]

{'train': './../.cache/model_1/data/train.jsonl',
 'test': './../.cache/model_1/data/test.jsonl',
 'valid': './../.cache/model_1/data/valid.jsonl'}

In [5]:
# Fine-Tuning with LoRA

lora = LORA(config={"train": True, "adapter_file": adapter_file, "batch_size": 1, "lora_layers": 4})
lora.invoke(model_path=model_path, data=data_folder)

Fetching 11 files:   0%|          | 0/11 [00:00<?, ?it/s]

Total parameters 7242.158M
Trainable parameters 0.426M
Loading datasets
Training
Iter 1: Val loss 2.963, Val took 6.470s
Iter 10: Train loss 2.764, It/sec 3.263, Tokens/sec 400.394
Iter 20: Train loss 1.939, It/sec 2.886, Tokens/sec 315.745
Iter 30: Train loss 1.372, It/sec 2.367, Tokens/sec 287.100
Iter 40: Train loss 1.175, It/sec 2.911, Tokens/sec 341.749
Iter 50: Train loss 1.000, It/sec 2.846, Tokens/sec 329.331
Iter 60: Train loss 1.089, It/sec 2.673, Tokens/sec 351.728
Iter 70: Train loss 0.821, It/sec 2.842, Tokens/sec 327.104
Iter 80: Train loss 0.917, It/sec 2.981, Tokens/sec 341.962
Iter 90: Train loss 0.995, It/sec 3.085, Tokens/sec 368.974
Iter 100: Train loss 0.917, It/sec 3.084, Tokens/sec 341.085
Iter 100: Saved adapter weights to ./../.cache/model_1/adapters.npz.
Iter 110: Train loss 0.780, It/sec 2.580, Tokens/sec 300.529
Iter 120: Train loss 0.799, It/sec 3.061, Tokens/sec 334.239
Iter 130: Train loss 0.865, It/sec 2.656, Tokens/sec 321.920
Iter 140: Train loss 0.799

In [6]:
# Fuse the LoRA adapters with the base model and save the fine-tuned model

fuse = FUSE(config={"adapter_file": adapter_file})
fuse.invoke(model_path=model_path, save_path=save_model_path)

Fetching 11 files:   0%|          | 0/11 [00:00<?, ?it/s]

In [7]:
# Load the fine-tuned model and generate a response

model, tokenizer = utils.load(save_model_path)
generate(model=model, tokenizer=tokenizer, prompt="You are an text to SQL query translator. Users will ask you questions in English and you will generate a SQL query based on the provided SCHEMA.\nSCHEMA:\nCREATE TABLE table_13505192_3 (series_number INTEGER, season_number VARCHAR)\nUser:What is the series number for season episode 24?\nAssistant:")

'SELECT season_number FROM table_13505192_3 WHERE season_number = "24"'